In [1]:
%use @file[../djl.json]
import ai.djl.metric.*

In [2]:
val manager = NDManager.newBaseManager()

In [3]:
val dim = 200L
val numEnt = 4L
val numEdg = 2L
val entities = manager.randomUniform(-0.07f, 0.07f, Shape(numEnt, dim))
val edges = manager.randomUniform(-0.07f, 0.07f, Shape(numEdg, dim))
DISPLAY(entities)
DISPLAY(edges)

ND: (4, 200) cpu() float32
[ Exceed max print size ]

ND: (2, 200) cpu() float32
[ Exceed max print size ]

In [4]:
entities.get(0)

ND: (200) cpu() float32
[ Exceed max print size ]

In [5]:
edges.get(1)

ND: (200) cpu() float32
[ Exceed max print size ]

In [6]:
val input = manager.create(longArrayOf(2,0,1,2,1,3,0,0,1,0,1,2), Shape(4,3))
input

ND: (4, 3) cpu() int64
[[ 2,  0,  1],
 [ 2,  1,  3],
 [ 0,  0,  1],
 [ 0,  1,  2],
]


In [7]:
var v = manager.zeros(Shape(0))
for (i in 0 until input.size(0) ) { 
    val line0 = input.get(i).toLongArray()
    println(entities.get(line0[0]).add(edges.get(line0[1])).sub(entities.get(line0[2])))
    v = v.concat(entities.get(line0[0]).add(edges.get(line0[1])).sub(entities.get(line0[2])))
}
println(v.reshape(input.size(0), dim))

ND: (200) cpu() float32
[ Exceed max print size ]
ND: (200) cpu() float32
[ Exceed max print size ]
ND: (200) cpu() float32
[ Exceed max print size ]
ND: (200) cpu() float32
[ Exceed max print size ]
ND: (4, 200) cpu() float32
[ Exceed max print size ]


In [8]:
class MyTransE(val numEnt: Long, val numEdge: Long, val dim: Long ): AbstractBlock() {

    private val entities:  Parameter 
    private val edges: Parameter
    

    // outUnits: the number of outputs in this layer 
    // inUnits: the number of inputs in this layer
    init {
        entities = addParameter(
            Parameter.builder()
                .setName("entities")
                .setType(Parameter.Type.WEIGHT)
                .optShape(Shape(numEnt, dim))
                .build())
        edges = addParameter(
            Parameter.builder()
                .setName("edges")
                .setType(Parameter.Type.WEIGHT)
                .optShape(Shape(numEdge, dim))
                .build())
    }
    
    @Override
    override protected fun forwardInternal(
            parameterStore: ParameterStore ,
            inputs: NDList ,
            training: Boolean,
            params: PairList<String, Any>?): NDList {
        val input = inputs.singletonOrThrow();
        val device = input.getDevice();
        // Since we added the parameter, we can now access it from the parameter store
        val entitiesArr = parameterStore.getValue(entities, device, false)
        val edgesArr = parameterStore.getValue(edges, device, false)
        return transE(input, entitiesArr, edgesArr)
    }
    
    // Applies linear transformation
    fun transE(input: NDArray , entities: NDArray , edges: NDArray) : NDList {
        var v = manager.zeros(Shape(0))
        val inputs = input.reshape(input.size()/3, 3)
        for (i in 0 until input.size()/3 ) { 
            val line0 = inputs.get(i).toLongArray()
//    println(entities.get(line0[0]).add(edges.get(line0[1])).sub(entities.get(line0[2])))
            v = v.concat(entities.get(line0[0]).add(edges.get(line0[1])).sub(entities.get(line0[2])))
        }
        val ret = v.reshape(input.size()/3, dim)
//        println(ret)
        return NDList(ret)
    }
        
    @Override
    override fun getOutputShapes(inputs: Array<Shape>) : Array<Shape> {
        return arrayOf<Shape>(Shape(1, dim))
    }
}

In [9]:
val linear = MyTransE(numEnt, numEdg, dim) 
var params = linear.getParameters();
for (param in params) {
    println(param.getKey());
}

entities
edges


In [10]:
val input = manager.create(longArrayOf(2,0,1,2,1,3,0,0,1,0,1,2), Shape(4,3))
//println(input)
linear.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT)
linear.initialize(manager, DataType.FLOAT32, input.getShape())

val model = Model.newInstance("my-linear");
model.setBlock(linear);

val predictor = model.newPredictor(NoopTranslator());
predictor.predict(NDList(input)).singletonOrThrow()

ND: (4, 200) cpu() float32
[ Exceed max print size ]

In [11]:
val entities = model.block.getParameters().valueAt(0).getArray()
println(entities)
val edges = model.block.getParameters().valueAt(1).getArray()
println(edges)

//val weight = linearLayer.getParameters().valueAt(0).getArray();
//weight

entities: (4, 200) cpu() float32 hasGradient
[ Exceed max print size ]
edges: (2, 200) cpu() float32 hasGradient
[ Exceed max print size ]


In [12]:
val l2loss = Loss.l2Loss()
val lrt = Tracker.fixed(0.05f)
val sgd = Optimizer.sgd().setLearningRateTracker(lrt).build()

In [13]:
val config = DefaultTrainingConfig(l2loss)
    .optOptimizer(sgd) // Optimizer (loss function)
    .optDevices(manager.getEngine().getDevices(1)) // single GPU
    .addTrainingListeners(*TrainingListener.Defaults.logging()) // Logging

val trainer = model.newTrainer(config)

In [14]:
trainer.initialize(Shape(4, 3))

In [15]:
val metrics = Metrics()
trainer.setMetrics(metrics);

In [16]:
//val parameterStore = ParameterStore(manager, false);
for(i in 0..1000) {
trainer.newGradientCollector().use { gc ->
//    val entities = model.block.getParameters().valueAt(0).getArray()
//    println(entities)
//    val f0 = model.block.forward(parameterStore, NDList(input), true)
    val f0 = trainer.forward(NDList(input))
//    println(f0.singletonOrThrow().sum(intArrayOf(1)))
//    println(f0.singletonOrThrow().sum(intArrayOf(1)))
    val point = f0.singletonOrThrow()
//    println(NDArrays.pow(2,point) == point.pow(2))
//    println(NDArrays.pow(2,point))
//    println(point.pow(2))
//    println(point)
//    println(NDArrays.pow(2, point).sum(intArrayOf(1)).sqrt())
      val l = point.pow(2).sum(intArrayOf(1)).sqrt().sub(manager.create(floatArrayOf(0f,0f,1f,0f))).abs()
//      println(NDArrays.pow(2, point).sum(intArrayOf(1)).sqrt())
//    val l : NDArray = l2loss.evaluate( NDList(NDArrays.pow(2, point).sum(intArrayOf(1)).sqrt()), NDList(manager.create(floatArrayOf(0f,0f,1f,0f))))
//    println("l = $l")
if(i % 50 == 0) {
    println("l = $l")
    
}
    gc.backward(l)
    trainer.step()
//    println(model.block.getParameters().valueAt(0).getArray())
    gc.close()
}
}
    println(model.block.getParameters().valueAt(0).getArray())
    println(model.block.getParameters().valueAt(1).getArray())



l = ND: (4) cpu() float32
[0.245 , 0.2466, 0.7344, 0.2462]

l = ND: (4) cpu() float32
[0.0848, 0.08  , 0.2485, 0.0786]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.1191, 0.006 , 0.1144]

l = ND: (4) cpu() float32
[0.1839, 0.119

In [17]:
predictor.predict(NDList(input)).singletonOrThrow().pow(2).sum(intArrayOf(1)).sqrt()

ND: (4) cpu() float32
[0.1161, 0.0809, 0.756 , 0.0606]


In [18]:
val test = manager.create(longArrayOf(1,1,2))
println(predictor.predict(NDList(test)).singletonOrThrow())
predictor.predict(NDList(test)).singletonOrThrow().pow(2).sum().sqrt()

ND: (1, 200) cpu() float32
[ Exceed max print size ]


ND: () cpu() float32
0.8201
